In [99]:
import numpy as np
import matplotlib.pyplot as plt

In [127]:
class LogisticRegression:
    '''
    method:['gd', 'cd', 'bfgs', 'lbfgs', 'liblinear'] - https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions
    https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451
    alpha:float - if method is 'gd': Learning rate in case of Gradient descent
                  if method is 'cd' and regularization is 'elastic-net': Elastic-net alpha parameter
    regularization:[None, 'l1', 'l2'] - None: No regularization,
                                        'l1': L1 penalty = absolute sum of the weights
                                        'l2': L2 penalty = sum of squares of the weights
    fit_intercept:bool - To use the intercept term, if False, X is assumed to be centered
    penalty:float - a.k.a lambda, multiplier that controls 
    n_iters:int - number of iterations in case of Gradient Descent Method
    '''
    def __init__(self, method='gd', alpha=0.1, fit_intercept=False, regularization='l2', reg_intercept=False, penalty=1.0, n_iters=400):
        if regularization is None:
            self.reg = None
        else:
            regularization = regularization.lower()
            if regularization == 'l1' or regularization == 'lasso':
                self.reg = 'l1'
            elif regularization == 'l2' or regularization == 'ridge':
                self.reg = 'l2'
        method = method.lower()
        if method == 'gd':
            assert self.reg == 'l2' or self.reg is None, "Gradient descent method can only be used with L2 regularization or None"
        self.method = method
        self.penalty = penalty
        self.reg_intercept = reg_intercept
        self.fit_intercept = fit_intercept
        self.W = np.array([])
        self.n_iters = n_iters
        self.alpha = alpha

    @staticmethod
    def sigmoid(z):
        return 1 / (1+np.exp(-z))
        
    def train(self, X, y, return_history=False):
        '''
        X - (n_datapoints, n_features)
        y - (n_datapoints, 1)
        '''
        if self.fit_intercept:
            # To accomodate the bias / y-intercept term
            X = np.insert(X, 0, np.ones(X.shape[0]), axis=1)
        n_datapoints, n_features = X.shape
        assert y.size == n_datapoints, "X and y must have same number of rows"
        if y.ndim == 1:
            y = y.reshape(-1, 1)
        n_classes = len(np.unique(y))
        if n_classes > 2:
            self.mode = 'ovr'
            W = np.zeros((n_features, n_classes))
            cost = []
            # One vs rest strategy
            for c in range(n_classes):
                y_c = np.where(y == c, 1, 0)
                if return_history:
                    W_c, cost_c = self._algo(X, y_c, True)
                    W[:, c] = W_c.ravel()
                    cost.append(cost_c)
                else:
                    W[:, c] = self._algo(X, y_c, False).ravel()
        else:
            self.mode = 'binary'
            if return_history:
                W, cost = self._algo(X, y, True)
            else:
                W = self._algo(X, y, False)
        self.W = W
        if return_history:
            return cost
        return self

    def _algo(self, X, y, cost_history):
        n_datapoints, n_features = X.shape
        W = np.zeros((n_features, 1))
        cost = []
        # Using Gradient Descent
        if self.method == 'gd':
            if self.reg is None:
                if cost_history:
                    for i in range(self.n_iters):
                        h = self.sigmoid(X@W)
                        W -= self.alpha * (X.T @ (h - y)) / n_datapoints
                        cost.append((- y.T@np.log(h) - (1-y.T)@np.log(1-h)).item() / n_datapoints)
                else:  
                    for i in range(self.n_iters):
                        W -= self.alpha * (X.T @ (self.sigmoid(X@W) - y)) / n_datapoints
            elif self.reg == 'l1':
                print("Gradient Descent not implemented for L1")
                raise NotImplementedError
            elif self.reg == 'l2':
                if self.fit_intercept and not(self.reg_intercept):
                    if cost_history:
                        for i in range(self.n_iters):
                            h = self.sigmoid(X@W)
                            W[0] = W[0] - self.alpha * (X[:, 0] @ (h - y)) / n_datapoints
                            W[1:] = W[1:] - self.alpha * (X[:, 1:].T @ (h - y) + self.penalty * W[1:].sum()) / n_datapoints
                            cost.append((- y.T@np.log(h) - (1-y.T)@np.log(1-h) + self.penalty * (W**2).sum() / 2 ).item() / n_datapoints)
                    else:
                        for i in range(self.n_iters):
                            h = self.sigmoid(X@W)
                            W[0] = W[0] - self.alpha * (X[:, 0] @ (h - y)) / n_datapoints
                            W[1:] = W[1:] - self.alpha * (X[:, 1:].T @ (h - y) + self.penalty * W[1:].sum()) / n_datapoints
                else:
                    if cost_history:
                        for i in range(self.n_iters):
                            W = W - self.alpha * (X.T @ (self.sigmoid(X@W) - y) + self.penalty * W.sum()) / n_datapoints
                            cost.append((- y.T@np.log(h) - (1-y.T)@np.log(1-h) + self.penalty * (W**2).sum() / 2 ).item() / n_datapoints)
                    else:
                        for i in range(self.n_iters):
                            W = W - self.alpha * (X.T @ (self.sigmoid(X@W) - y) + self.penalty * W.sum()) / n_datapoints
        else:
            print("Unknown method, might not be implemented yet, training failed")
            raise NotImplementedError
        if cost_history:
            return W, cost
        return W
        
    def predict(self, X, return_prob=False, threshold=0.5):
        if self.fit_intercept:
            X = np.insert(X, 0, np.ones(X.shape[0]), axis=1)
        prob = self.sigmoid(X@self.W)
        if self.mode == 'ovr':
            pred = prob.argmax(axis=1).ravel()
        elif self.mode == 'binary':
            pred = np.where(prob > threshold, 1, 0).ravel()
        else:
            raise ValueError
        if return_prob:
            return pred, np.around(prob, decimals=2)
        return pred

    @staticmethod
    def score(y, y_pred):
        return 100 * (y==y_pred).sum() / len(y)

In [180]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, plot_confusion_matrix
def pipe(X, y, models, normalize=False, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, shuffle=True, random_state=3)
    for k in models:
        if 'sk' in k:
            models[k].fit(X_train, y_train)
        else:
            if normalize:
                X_train = Normalizer(method=norm_method).normalize(X_train)
                X_test = Normalizer(method=norm_method).normalize(X_test)
            models[k].train(X_train, y_train)

        y_pred = models[k].predict(X)
        print("-"*20, k, "-"*20)
        print("sklearn Accuracy scores ---------")
        print("Test Accuracy score = ", accuracy_score(y_test, models[k].predict(X_test)))
        print("Whole dataset Accuracy score = ", accuracy_score(y, y_pred))
        print("My Accuracy scores ---------")
        print("Test Accuracy score = ", LogisticRegression().score(y_test, models[k].predict(X_test)))
        print("Whole dataset Accuracy score = ", LogisticRegression().score(y, y_pred))
    return models

In [155]:
def mapFeature(X1, X2, deg=6, fit_intercept=False):
    if hasattr(X1, "__iter__"):
        out = np.zeros((len(X1), deg))
    else:
        out = np.zeros((1, deg))
    for i in range(1, deg+1):
        for j in range(i+1):
            out[:, i-1] = X1**(i-j) * X2**j
    if fit_intercept:
        return np.insert(out, 0, np.ones(out.shape[0]), axis=1)
    return out

In [181]:
from sklearn import linear_model

In [182]:
alpha = .1
pen = 1.0
reg = 'l2'
models = {
    'skLR': linear_model.LogisticRegression(C=pen, penalty=str(reg).lower(), fit_intercept=True),
    'myLR': LogisticRegression(alpha=alpha, regularization=reg, fit_intercept=True, penalty=pen),
    'myLR_reg_int': LogisticRegression(alpha=alpha, fit_intercept=True, regularization=reg, reg_intercept=True, penalty=pen),
    'skLR_no_int': linear_model.LogisticRegression(penalty=str(reg).lower(), fit_intercept=False, C=pen),
    'myLR_no_int': LogisticRegression(alpha=alpha, regularization=reg, fit_intercept=False, penalty=pen),
}
models = pipe(X, y, models)

-------------------- skLR --------------------
sklearn Accuracy scores ---------
Test Accuracy score =  0.9
Whole dataset Accuracy score =  0.9666666666666667
My Accuracy scores ---------
Test Accuracy score =  90.0
Whole dataset Accuracy score =  96.66666666666667
-------------------- myLR --------------------
sklearn Accuracy scores ---------
Test Accuracy score =  0.9666666666666667
Whole dataset Accuracy score =  0.9666666666666667
My Accuracy scores ---------
Test Accuracy score =  96.66666666666667
Whole dataset Accuracy score =  96.66666666666667
-------------------- myLR_reg_int --------------------
sklearn Accuracy scores ---------
Test Accuracy score =  0.9333333333333333
Whole dataset Accuracy score =  0.96
My Accuracy scores ---------
Test Accuracy score =  93.33333333333333
Whole dataset Accuracy score =  96.0
-------------------- skLR_no_int --------------------
sklearn Accuracy scores ---------
Test Accuracy score =  0.9666666666666667
Whole dataset Accuracy score =  0.9